### Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [130]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [131]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)   #tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [132]:
train_doc = np.random.permutation(open('C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/val.csv').readlines())
batch_size = 10 #70 #30 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [133]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,3,6,9,12,15,18,21,24,27,29] #[0,2,5,7,10,12,15,17,20,22,25,27,29]#[0,2,4,7,10,12,15,16,18,20,22,25,27,28,29] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,11,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1] == 160:
                        resized_image = imresize(image[:,20:140,:],(120,120)).astype(np.float32)
                    else:
                        resized_image = imresize(image,(120,120)).astype(np.float32)
                        
                    
                    batch_data[folder,idx,:,:,0] = resized_image[:,:,0] - 104 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_image[:,:,1] - 117 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_image[:,:,2] - 123 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

            
# write the code for the remaining data points which are left after full batches      


            if (len(t)%batch_size) != 0:
                batch_data = np.zeros((len(t)%batch_size,11,120,120,3))
                batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        resized_image = imresize(image[:,20:140,:],(120,120)).astype(np.float32)
                    else:
                        resized_image = imresize(image,(120,120)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = resized_image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = resized_image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = resized_image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [134]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/train'
val_path = 'C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 #50 #70 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [135]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

from keras.applications.vgg16 import VGG16

base = VGG16(include_top=False, weights='imagenet', input_shape=(120,120,3))
x = base.output
x = Flatten()(x)

features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base.input, outputs=features)
    
for layer in base.layers:
    layer.trainable = False
        
model2d = Sequential()
model2d.add(TimeDistributed(conv_model, input_shape=(11,120,120,3)))
model2d.add(GRU(32, return_sequences=True))
model2d.add(GRU(16))
model2d.add(Dropout(0.5))
model2d.add(Dense(8, activation='relu'))
model2d.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [136]:
from keras.optimizers import Adam

optimiser =Adam(learning_rate=0.001) #write your optimizer

model2d.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2d.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_10 (TimeDis (None, 11, 64)            15009664  
_________________________________________________________________
gru_19 (GRU)                 (None, 11, 32)            9312      
_________________________________________________________________
gru_20 (GRU)                 (None, 16)                2352      
_________________________________________________________________
dropout_10 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_31 (Dense)             (None, 5)                 45        
Total params: 15,021,509
Trainable params: 306,821
Non-trainable params: 14,714,688
___________________________________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [137]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [138]:
model_name = 'model2d' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [139]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [140]:
model2d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/val ; batch size = 10
Source path =  C:/Users/induj/ML_and_AI/Case_Studies/Deep Learning Project/My Code/Project_data/Project_data/train ; batch size = 10


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipy

67/67 [==============================] - 511s 8s/step - loss: 1.2770 - categorical_accuracy: 0.3781 - val_loss: 1.4513 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model2d_2020-05-0415_51_27.088527/model-00001-1.52098-0.37813-1.45128-0.30000.h5
Epoch 2/20
67/67 [==============================] - 519s 8s/step - loss: 0.9985 - categorical_accuracy: 0.5278 - val_loss: 1.4747 - val_categorical_accuracy: 0.3800

Epoch 00002: saving model to model2d_2020-05-0415_51_27.088527/model-00002-1.28784-0.52778-1.47468-0.38000.h5
Epoch 3/20
67/67 [==============================] - 540s 8s/step - loss: 1.0808 - categorical_accuracy: 0.4989 - val_loss: 1.0700 - val_categorical_accuracy: 0.5600

Epoch 00003: saving model to model2d_2020-05-0415_51_27.088527/model-00003-1.23607-0.49886-1.06995-0.56000.h5
Epoch 4/20
67/67 [==============================] - 510s 8s/step - loss: 0.8991 - categorical_accuracy: 0.5440 - val_loss: 1.4672 - val_categorical_accuracy: 0.3600

Epoch 00004: savi